In [1]:
import os

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  # Arrange GPU devices starting from 0
os.environ["CUDA_VISIBLE_DEVICES"]= "3"

In [2]:
import torch

from utils import set_seed, print_trainable_parameters

from datasets import load_dataset
from transformers import (AutoTokenizer, AutoModelForCausalLM,
                          BitsAndBytesConfig, Trainer,
                          TrainingArguments, DataCollatorForLanguageModeling)

from peft import prepare_model_for_kbit_training, LoraConfig, PeftModel

/home/nlplab11/anaconda3/envs/jyp/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/nlplab11/anaconda3/envs/jyp/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda113.so
CUDA SETUP: CUDA runtime path found: /home/nlplab11/anaconda3/envs/jyp/lib/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 113
CUDA SETUP: Loading binary /home/nlplab11/anaconda3/envs/jyp/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda113.so...


/home/nlplab11/anaconda3/envs/jyp/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: Found duplicate ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] files: {PosixPath('/home/nlplab11/anaconda3/envs/jyp/lib/libcudart.so.11.0'), PosixPath('/home/nlplab11/anaconda3/envs/jyp/lib/libcudart.so')}.. We'll flip a coin and try one of these, in order to fail forward.
Either way, this might cause trouble in the future:
If you get `CUDA error: invalid device function` errors, the above might be the cause and the solution is to make sure only one ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] in the paths that we search based on your env.
  warn(msg)


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print('Device:', device)
print('Current cuda device:', torch.cuda.current_device())
print('Count of using GPUs:', torch.cuda.device_count())

Device: cuda
Current cuda device: 0
Count of using GPUs: 1


In [4]:
# BitsAndBytes Config for QLoRA
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [5]:
checkpoint = 'beomi/kollama-13b'

In [6]:
# LOAD TOKENIZER
tokenizer = AutoTokenizer.from_pretrained(
    checkpoint,
    use_fast=False,
    truncation_side='left',
    )

In [7]:
model_checkpoint = 'checkpoint-2500'

In [8]:
model = AutoModelForCausalLM.from_pretrained(checkpoint,
                                            quantization_config=bnb_config,
                                            device_map='auto')

# PREPARE MODEL FOR K BIT TRAINING
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

# LoRA Config
config = LoraConfig.from_json_file(f'outputs/{model_checkpoint}/adapter_config.json')

model = PeftModel.from_pretrained(model, model_id=f'outputs/{model_checkpoint}/')

Loading checkpoint shards: 100%|██████████| 28/28 [00:05<00:00,  5.25it/s]


In [9]:
!nvidia-smi

Thu Jun 29 10:41:44 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.103.01   Driver Version: 470.103.01   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0 Off |                  N/A |
| 40%   30C    P8    20W / 350W |      8MiB / 24268MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  On   | 00000000:21:00.0 Off |                  N/A |
| 39%   

In [10]:
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(52000, 5120, padding_idx=0)
        (layers): ModuleList(
          (0): LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): Linear4bit(
                in_features=5120, out_features=5120, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=5120, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=5120, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
              (v_proj): 

In [11]:
prompt = [
    'Below is an instruction that describes a task, paired with an input that provides further context.',
    '아래는 작업을 설명하는 명령어와 추가적 맥락을 제공하는 입력이 짝을 이루는 예제입니다.\n',
    'Write a response that appropriately completes the request.',
    '요청을 적절히 완료하는 응답을 작성하세요.\n',
    '### Instruction(명령어):',
    '아이의 요구사항을 그림 생성을 위한 프롬프트를 번역하세요.\n',
    '### Input(입력):',
    '아이: 북극을 가로지르는 트렉을 그려줘. 눈보라가 일어나고 있을 때의 그림이면 좋겠어. 그리고 그림에는 흰색 계열을 많이 사용해줘.\n',
    '### Response(응답):'
    ]

inputs = tokenizer('\n'.join(prompt), return_tensors="pt")

In [13]:
print('\n'.join(prompt))

Below is an instruction that describes a task, paired with an input that provides further context.
아래는 작업을 설명하는 명령어와 추가적 맥락을 제공하는 입력이 짝을 이루는 예제입니다.

Write a response that appropriately completes the request.
요청을 적절히 완료하는 응답을 작성하세요.

### Instruction(명령어):
아이의 요구사항을 그림 생성을 위한 프롬프트를 번역하세요.

### Input(입력):
아이: 북극을 가로지르는 트렉을 그려줘. 눈보라가 일어나고 있을 때의 그림이면 좋겠어. 그리고 그림에는 흰색 계열을 많이 사용해줘.

### Response(응답):


In [12]:
with torch.no_grad():
    outputs = model.generate(input_ids=inputs['input_ids'].to('cuda'),
                             max_length=256,
                            #  no_repeat_ngram_size=2,
                             early_stopping=True)
    outputs = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]
    print(outputs)

아이: 북극을 가로지르는 트렉을 그려줘. 눈보라가 일어나고 있을 때의 그림이면 좋겠어. 그리고 그림에는 흰색 계열을 많이 사용해줘.
Model: a bearing painting of a magic sunset, smoke, black and white, and bright colors, highly detailed, --ar 16:9
아이아이, 그림에 물결이 물결도 넣어줘, 그리고 물줄기가 물결을 따라줘."
모델: A beautiful paintering a sandy spark, with a large sky, moon, w


In [13]:
prompt = [
    '아이: 북극을 가로지르는 트렉을 그려줘. 눈보라가 일어나고 있을 때의 그림이면 좋겠어. 그리고 그림에는 흰색 계열을 많이 사용해줘.',
    'Model: A glacier snowman, blue colorful, black, --w 1920 --h 1080',
    '아이: 그리고 그 위에 잉크를 뿌려봐. 그게 왁스 프린트 같은 느낌이 나면 좋겠어.',
    'Model:']

inputs = tokenizer('\n'.join(prompt), return_tensors="pt")

In [14]:
with torch.no_grad():
    outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_length=256, no_repeat_ngram_size=2)
    outputs = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]
    print(outputs)

아이: 북극을 가로지르는 트렉을 그려줘. 눈보라가 일어나고 있을 때의 그림이면 좋겠어. 그리고 그림에는 흰색 계열을 많이 사용해줘.
Model: A glacier snowman, blue colorful, black, --w 1920 --h 1080
아이: 그리고 그 위에 잉크를 뿌려봐. 그게 왁스 프린트 같은 느낌이 나면 좋겠어.
Model: Cyberdeck smoke, red, pink, photo, cinematic, dark, spark, detailed, octane render, light --ar 16:9
아이가 움직이면서 그려줄 수 있을까?
모델: Blushmurm snovit, golden, pattern, volumetric, flight, highly detailed --aspect 1620
아. 그러면 그림에 물결이 살짝 비춰지는 느낌도 있으면 좋을까? 그리고 물결이나 물방울이 있으면 더 좋겠지. --Model -- Blanchetta, wave, shallow, render --light --shoth, water, droplets, moonlights, and sunsets --uplighting --stop 16

아이들: 그림이 좀 더 생동감 있게


In [15]:
prompt = [
    '아이: 사막에서 별문을 연상시키는 문을 그려봐. 그리고 판타지 잡지 같은 느낌도 넣어줘.',
    'Model:']

inputs = tokenizer('\n'.join(prompt), return_tensors="pt")

with torch.no_grad():
    outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_length=256, no_repeat_ngram_size=2)
    outputs = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]
    print(outputs)

아이: 사막에서 별문을 연상시키는 문을 그려봐. 그리고 판타지 잡지 같은 느낌도 넣어줘.
Model: a spaceship with a starry sunset, smoke, ultra-detail, cinematic, photorealistic, octane render, --ar 16:9
아이 : 그리고 그림이 좀 더 생동감 있게 보이면 좋겠어. 그래서 더 좋을까?
빛이 있는 느낌도 있으면 좋잖아. 그런데 그림에 좀 무서운 느낌이 나게 해주세요. 좀 좀 어두운 느낌이 나면 좋겠는데요. --w 1620 --h 1080
아 그리고 배경은 좀 어둡게 해줘, 그리고 좀 밝게 해서 더 크게 그려줘
미감:
초록이 많은 느낌을 내면 좋지만, 좀 크게 해줄 수 있을까? 그리고 빛이 좀 비춰지는 느낌도 나도록 해주세요. -미감도 좀 크도록,
자연적인 느낌도 좀 나게하는 느낌으로 해줘요. –w 1080 --lighting, brighter lighting, light effect, and ambiance, feeling. The background is blurry, more larger, with luminous, fluffy, volumetric,, but, too large, –ar 1080 -light
